# About
In recent years, Cloud Model 1 (CM1; http://www2.mmm.ucar.edu/people/bryan/cm1/) has become a very popular tool for performing idealized studies of atmospheric phenomena. There exists very little support for computing trajectories using CM1 output, which are usually necessary to understand the processes of the atmospheric phenomena of interest. Natively, CM1 only supports 'online' forward trajectories in 2D simulations and in 3D simulation without terrain. I wrote this script because there are no adequate tools available to compute highly customizable 'offline' trajectories in simulations with terrain. This script is intended to be easily customizable.

Notes:

* Can compute backward or forward trajectories (Default is backward, but can be forward with simple changes to "Calculate Trajectories" block)
* Written to work with 3D model output (can be modified to work with 2D output)
* Will work with or without terrain
* Initial location, number, and density of parcels can be easily specified in "Initialize Parcels" block
* Uses xarray and Dask to distribute memory and calculation across multiple processors
* With modifications, can be used with WRF output (several others have already done so)
* Comments that say "set by user" are specific to model output and desired trajectories

# Load Modules

In [1]:
import os, sys
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate
import time

%config IPCompleter.greedy=True

# Read in CM1 Output

* User must insert path to data
    * If model output is one file use ***xr.open_dataset***
    * If model output is in multiple files use ***xr.openmfdataset***


In [2]:
#Use xarray to open model output and specify chunking if data set is large (set by user)
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/steenburgh-group8/tom/cm1/output/15ms_1500m_tug.nc', chunks={'nk': 1})

#Get model output dimensions
num_x = ds.nx #Number of gridpoints in x
num_y = ds.ny #Number of gridpoints in y
num_z = ds.nz #Number of gridpoints in z

x = np.arange(0,num_x,1)
y = np.arange(0,num_y,1)
z = np.arange(0,num_z,1)

# Create Dask Cluster

In [3]:
#Option to use multiple processors and threads (set by user)
from dask.distributed import Client, LocalCluster
c = LocalCluster(n_workers=10, threads_per_worker=4)
client = Client(c)
client

Client Scheduler: tcp://127.0.0.1:39688 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 10 Cores: 40 Memory: 96.49 GB


# Initialize Parcels

User must enter desired trajectory characteristics

In [4]:
#Number of parcels in vertical (can be more than number of vertical levels; set by user) 
num_seeds_z = 10

#Number of parcels in y (set by user) 
num_seeds_y = 10

#Number of time steps to run trajectories back (set by user) 
time_steps = 50

#Time step to start backward trajectories at (set by user) 
start_time_step = 210

#Variable to record at each parcel's location throughout trajectory (code can be easily modified to add more; set by user) 
var_name1 = 'th'

#Set as 'Y' or 'N' for 'yes' or 'no' if the u, v, and w model output is on the staggered grid 
#(unless you have interpolated u, v, and w to the scalar grid, they are most likely on the staggered grid (set by user)
staggered = 'N'

**Model output info**

In [5]:
#Horizontal resolution of model output (meters)
hor_resolution = (ds.xf[1].values-ds.xf[0].values)*1000

#Vertical resolution of model output (meters). Changes in x and y, if there is terrain, and z, if grid is stretched.
vert_resolution = ds.zh[0,1:,:,:].values-ds.zh[0,:-1,:,:].values 
                  
#Model output time step length (seconds)
time_step_length = (ds.time[1].values - ds.time[0].values)/np.timedelta64(1, 's')

**Create empty arrays to store x, y, and z positions of parcels**

In [6]:
xpos = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #x-location (grid points on staggered grid)
ypos = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #y-location (grid points on staggered grid)
zpos = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #z-location (grid points on staggered grid)
zpos_heightASL = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #Height above sea level (meters)
zpos_vert_res = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #Vertical grid spacing at parcel location (meters)
variable1 = np.zeros((time_steps, num_seeds_z, num_seeds_y)) #User specified variable to track

**Initial location of parcels in gridpoints, specifically on the staggered grid (set by user). Initializes an array of parcels in the the y-z domain (modification necessary for x-dimension or 3D array of parcels)**

In [7]:
#x-position
xpos[0,:,:] = 1000 #This example initializes all seeds at same x-position (1000th x-grpt, set by user)

#y-position   
for i in range(num_seeds_y):
    ypos[0,:,i] = 4*i #This example initializes seeds evenly in y-dimension (0th, 4th, 8th, etc. y-grpt; set by user)

#z-position
for i in range(num_seeds_z):
    zpos[0,i,:] = i #This example initializes seeds evenly starting in z-dimension (0th, 1st, 2nd, etc., z-grpt; set by user)

## Determine Initial Height of Parcels Above Sea Level
Use the height of the models levels (meters above sea level) to convert from terrain following grid points to height above seal level.

In [8]:
#Get height of vertical coordinates (scalar grid)
zh = ds.zh[0,:,:,:].values

#Create list of initial coordinates to get height (must subtract 0.5 grdpt from each location because zh is on scalar grid)
xloc = (xpos[0,:,:]-0.5).flatten()
yloc = (ypos[0,:,:]-0.5).flatten()
zloc = (zpos[0,:,:]-0.5).flatten()
coord_height = []
for i in range(len(xloc)):
    coord_height.append((zloc[i], yloc[i], xloc[i]))

#Get the actual inital height of the parcels in meters above sea level
zpos_heightASL[0,:,:] = np.reshape(interpolate.interpn((z,y,x), zh, coord_height, method='linear', bounds_error=False, fill_value= 0), (num_seeds_z, num_seeds_y))

# Calculate Trajectories
Unless user is changing trajectories from backwards to forwards, nothing should be changed here.

In [9]:
#Loop over all time steps and compute trajectory
for t in range(time_steps-1):
    
    start = time.time() #Timer
    
    #Get model data
    u = ds.uinterp[start_time_step-t,:,:,:].values
    v = ds.vinterp[start_time_step-t,:,:,:].values
    w = ds.winterp[start_time_step-t,:,:,:].values
    var1 = getattr(ds,var_name1)[start_time_step-t,:,:,:].values
        
    #Get surface height grid (set to zero if no terrain)
    try:
        zs = np.array(ds.zs[0,:,:])
    except:
        zs = np.zeros((ds.ny, ds.nx))  
        
        
    ############## Generate coordinates for interpolations ###############

    #x, y, and z on staggered and scalar grids
    xloc = np.copy(xpos[t,:,:]-0.5).flatten()
    xloc_stag = np.copy(xpos[t,:,:]).flatten()
    yloc = np.copy(ypos[t,:,:]-0.5).flatten()
    yloc_stag = np.copy(ypos[t,:,:]).flatten()
    zloc = np.copy(zpos[t,:,:]-0.5).flatten()
    zloc_stag = np.copy(zpos[t,:,:]).flatten()

    #If u, v, and w are staggered, generate three staggered sets of coordinates:
    #    1) u-grid (staggered in x)
    #    2) v-grid (staggered in y)
    #    3) w-grid (staggered in z)
    
    if staggered == 'Y':
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc_stag[i])) 
            coord_v.append((zloc[i], yloc_stag[i], xloc[i])) 
            coord_w.append((zloc_stag[i], yloc[i], xloc[i])) 
    
    #If not, generate scalar coordinates
    else: 
        coord_u = []
        coord_v = []
        coord_w = []
        for i in range(len(xloc)):
            coord_u.append((zloc[i], yloc[i], xloc[i])) 
            coord_v.append((zloc[i], yloc[i], xloc[i])) 
            coord_w.append((zloc[i], yloc[i], xloc[i])) 
    
    #Scalar coordinates for all other variables
    coord = []
    for i in range(len(xloc)):
        coord.append((zloc[i], yloc[i], xloc[i])) 
    
    ##########################################################################################################   
    ########################## Integrate to determine parcel's new location ##################################
    ##########################################################################################################   

    
    #########################   Calc new xpos in grdpts above surface  #######################################
    xpos[t+1,:,:] = xpos[t,:,:] - np.reshape(interpolate.interpn((z,y,x), u, coord_u, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))

    #########################   Calc new ypos in grdpts above surface  #######################################
    ypos[t+1,:,:]  = ypos[t,:,:] - np.reshape(interpolate.interpn((z,y,x), v, coord_v, method='linear', bounds_error=False, fill_value=np.nan)*time_step_length/hor_resolution, (num_seeds_z, num_seeds_y))

    #########################   Calc new zpos in meters above sea level ######################################
    zpos_heightASL[t+1,:,:]  = zpos_heightASL[t,:,:] - np.reshape(interpolate.interpn((z,y,x), w, coord_w, method='linear', bounds_error=False, fill_value= 0)*time_step_length, (num_seeds_z, num_seeds_y))

    ############# Convert zpos from meters above sea level to gridpts abve surface for interpolation #########
    #Get vertical grid spacing at each parcel's location
    zpos_vert_res[t,:,:] = np.reshape(interpolate.interpn((z[:-1],y,x), vert_resolution, coord, method='linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))

    #Get surface height at each parcel's location (scalar)
    xloc = np.copy(xpos[t+1,:,:]-0.5).flatten()
    yloc = np.copy(ypos[t+1,:,:]-0.5).flatten()
    coord_zs = []
    for i in range(len(xloc)):
        coord_zs.append((yloc[i], xloc[i]))
    surface_height = np.reshape(interpolate.interpn((y,x), zs, coord_zs, method='linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))
    
    #Calculate zpos in grdpts above surface
    zpos[t+1,:,:] = np.reshape((zpos_heightASL[t+1,:,:].flatten()-surface_height.flatten())/zpos_vert_res[t,:,:].flatten(), (num_seeds_z, num_seeds_y))
    
    ##########################################################################################################
    
    
    #Prevent parcels from going into the ground
    if staggered == 'Y':
        zpos = zpos.clip(min=0.5)
    else:
        zpos = zpos.clip(min=0)
    
    #Calculate value of variable at each parcel's location
    variable1[t,:,:] = np.reshape(interpolate.interpn((z,y,x), var1, coord, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))  
    
    #Timer
    stop = time.time()
    print("Integration {:01d} took {:.2f} seconds".format(t, stop-start))
    
    

Integration 0 took 12.42 seconds
Integration 1 took 12.84 seconds
Integration 2 took 12.15 seconds
Integration 3 took 11.73 seconds
Integration 4 took 13.21 seconds
Integration 5 took 12.78 seconds
Integration 6 took 12.89 seconds
Integration 7 took 12.28 seconds
Integration 8 took 11.67 seconds
Integration 9 took 12.73 seconds
Integration 10 took 11.60 seconds
Integration 11 took 12.93 seconds
Integration 12 took 12.28 seconds
Integration 13 took 12.00 seconds
Integration 14 took 13.56 seconds
Integration 15 took 11.78 seconds
Integration 16 took 12.47 seconds
Integration 17 took 12.29 seconds
Integration 18 took 12.11 seconds
Integration 19 took 12.57 seconds
Integration 20 took 13.05 seconds
Integration 21 took 12.94 seconds
Integration 22 took 12.16 seconds
Integration 23 took 12.80 seconds
Integration 24 took 12.83 seconds
Integration 25 took 11.83 seconds
Integration 26 took 12.41 seconds
Integration 27 took 12.38 seconds
Integration 28 took 11.93 seconds
Integration 29 took 12.4

Get variable data for final time step

In [10]:
t = time_steps-1

#Get get x, y, and z positions from scalar grid
xloc = np.copy(xpos[t,:,:]-0.5).flatten()
yloc = np.copy(ypos[t,:,:]-0.5).flatten()
zloc = np.copy(zpos[t,:,:]-0.5).flatten()
coord = []
for i in range(len(xloc)):
    coord.append((zloc[i], yloc[i], xloc[i])) 

#Variables
variable1[t,:,:] = np.reshape(interpolate.interpn((z,y,x), var1, coord, method = 'linear', bounds_error=False, fill_value= np.nan), (num_seeds_z, num_seeds_y))

# Save Trajectory Data
The x, y, and z positions and user-specified variable values are saved in 3D numpy arrays. The first dimension is time and the other two are the positions and values of variables of all the parcels at that specifc time.

In [11]:
np.save('trajectory_arrays/xpos', xpos)
np.save('trajectory_arrays/ypos', ypos)
np.save('trajectory_arrays/zpos', zpos_heightASL)
np.save('trajectory_arrays/%s' %var_name1, variable1)